# Centralized Learning

In [1]:
from collections import OrderedDict
from typing import List, Tuple
import time
import flwr as fl
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from flwr.server.strategy import FedAvg
from torchvision.models._api import Weights
from torchvision.transforms import transforms
from flwr.common import Metrics, parameters_to_ndarrays, ndarrays_to_parameters, FitRes
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

from tinysmpc import VirtualMachine, PrivateScalar
from tinysmpc.fixed_point import fixed_point, float_point

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}")

Training on cpu using PyTorch 1.13.1+cpu and Flower 1.1.0


In [2]:
CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
NUM_CLIENTS = 5

In [3]:

BATCH_SIZE = 32


def load_datasets():
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=False, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=False, transform=transform)

    # Split training set into 10 partitions to simulate the individual dataset
    partition_size = len(trainset) // NUM_CLIENTS
    print(f"Train Size {len(trainset)}")
    print(f"Partition Size {partition_size}")
    print(f"Test Size {len(testset)}")
    lengths = [partition_size] * NUM_CLIENTS
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets()

Train Size 50000
Partition Size 10000
Test Size 10000


In [4]:
# images, labels = next(iter(trainloaders[0]))
#
# # Reshape and convert images to a NumPy array
# # matplotlib requires images with the shape (height, width, 3)
# images = images.permute(0, 2, 3, 1).numpy()
# # Denormalize
# images = images / 2 + 0.5
#
# # Create a figure and a grid of subplots
# fig, axs = plt.subplots(4, 8, figsize=(12, 6))
#
# # Loop over the images and plot them
# for i, ax in enumerate(axs.flat):
#     ax.imshow(images[i])
#     ax.set_title(CLASSES[labels[i]])
#     ax.axis('off')
#
# # Show the plot
# fig.tight_layout()
# plt.show()

In [5]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    # net.train()

    times = []
    for _ in range(epochs):
        # measure start time
        torch.cuda.synchronize()
        start_epoch = time.time()
        # stat training
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
        # measure end time
        torch.cuda.synchronize()
        end_epoch = time.time()
        elapsed = end_epoch - start_epoch
        times.append(elapsed)

    avg_time = sum(times)/epochs
    return avg_time

def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    # net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [7]:
trainloader = trainloaders[0]
valloader = valloaders[0]
net = Net().to(DEVICE)

train(net, trainloader, 5, True)

loss, accuracy = test(net, testloader)
print(f"Final test set performance:\n\tloss {loss}\n\taccuracy {accuracy}")

Epoch 1: train loss:  0.062195029109716415, accuracy: 0.26311111111111113, TIME TAKEN: 5.108340740203857
Epoch 2: train loss:  0.05208200216293335, accuracy: 0.3877777777777778, TIME TAKEN: 10.019147396087646
Epoch 3: train loss:  0.047951873391866684, accuracy: 0.43433333333333335, TIME TAKEN: 14.927428245544434
Epoch 4: train loss:  0.044960908591747284, accuracy: 0.47588888888888886, TIME TAKEN: 20.255348682403564
Epoch 5: train loss:  0.04266203194856644, accuracy: 0.5118888888888888, TIME TAKEN: 25.316556930541992
Final test set performance:
	loss 0.04581377928256988
	accuracy 0.4737


In [8]:
print(net.parameters)

<bound method Module.parameters of Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>


In [9]:
net.state_dict()

OrderedDict([('conv1.weight',
              tensor([[[[ 0.1753,  0.1473,  0.0541,  0.1618, -0.0029],
                        [-0.0410,  0.1098,  0.1390,  0.1130,  0.0587],
                        [ 0.1716,  0.0840, -0.0055,  0.0209,  0.0132],
                        [ 0.1452,  0.1406,  0.0377,  0.0542,  0.0675],
                        [ 0.1550,  0.1388, -0.0691,  0.0921,  0.0469]],
              
                       [[ 0.0095, -0.0393,  0.0235, -0.0725, -0.0909],
                        [ 0.1215,  0.0278,  0.0321, -0.0635,  0.0710],
                        [ 0.1387, -0.0045, -0.0255, -0.0351, -0.0810],
                        [ 0.0891, -0.0052, -0.1069, -0.0206, -0.0492],
                        [-0.0430, -0.0009, -0.0459,  0.0498,  0.0006]],
              
                       [[-0.0678, -0.0633, -0.0884, -0.0864, -0.1279],
                        [-0.0250,  0.0730, -0.1116, -0.2130, -0.1432],
                        [-0.0783,  0.0217, -0.0900, -0.0814, -0.0203],
               

In [10]:
parameters = [val.cpu().numpy() for _, val in net.state_dict().items()]
len(parameters)

10

In [11]:
net.state_dict()['fc3.weight']

tensor([[ 0.0317, -0.1328, -0.0573,  0.1033,  0.0970,  0.0412,  0.0138,  0.0412,
          0.1209,  0.1548,  0.0555,  0.1102, -0.1893,  0.0236,  0.0269,  0.0907,
          0.0624,  0.0033, -0.0274, -0.0588, -0.0124, -0.0995, -0.0053,  0.0345,
         -0.0682, -0.1337, -0.0054,  0.0905, -0.0977, -0.1303,  0.1045,  0.0317,
         -0.0897,  0.0233, -0.0565, -0.1304,  0.0327, -0.0551,  0.0885, -0.1421,
         -0.0848,  0.0257, -0.0420, -0.0682,  0.0899,  0.0709,  0.1061, -0.0587,
         -0.1318,  0.0675,  0.0999, -0.0287, -0.1010, -0.0295, -0.1061, -0.0033,
          0.0095, -0.0724, -0.0477,  0.1245, -0.1303, -0.0031, -0.0293,  0.0293,
          0.0902, -0.0089, -0.1365,  0.0121,  0.1358,  0.0795, -0.1143, -0.0453,
         -0.0589, -0.1328,  0.0537, -0.1415,  0.0052, -0.0370,  0.1116,  0.1462,
         -0.0348, -0.0206,  0.1128, -0.0089],
        [-0.0507,  0.0865, -0.0442,  0.0604, -0.0381, -0.0038,  0.0432,  0.0236,
         -0.2108, -0.0847, -0.1643,  0.0431, -0.0559,  0.0102, 

In [12]:
numpy_weights = net.state_dict()['fc3.weight'].cpu().detach().numpy()
print(numpy_weights.shape)

(10, 84)


In [13]:
print(numpy_weights[:3].shape)

(3, 84)


In [14]:
from flwr.common import ndarray_to_bytes

tensors = [ndarray_to_bytes(ndarray) for ndarray in net.state_dict()['fc3.weight']]
tensors

[b"\x93NUMPY\x01\x00v\x00{'descr': '<f4', 'fortran_order': False, 'shape': (84,), }                                                           \n5\xfd\x01=\x1a\xf7\x07\xbe:\x8aj\xbd\xa8x\xd3=\x93\x9a\xc6=\xe0\xb5(=\x96\xc7a<\xe0\xf0(=\x19\xab\xf7=\xb7\x90\x1e>\x81jc=\xc6\xbf\xe1=\xb6\xd4A\xbeD5\xc1<P\x9b\xdc<\xe1\xb1\xb9=\x90\xb6\x7f=\xe9\xedV;\x9b\xb9\xe0\xbc|\xfap\xbdF\\J\xbc\xb7\xc5\xcb\xbd\xef\x7f\xac\xbbn[\r=\x95\x92\x8b\xbd\xf0\xf4\x08\xbe\x05c\xb2\xbb\xdaG\xb9=\x96\xff\xc7\xbdIe\x05\xbe.\xef\xd5=e\xe4\x01=H\xa3\xb7\xbd\x8c\x1c\xbf<o\x8fg\xbd\xc9\x8c\x05\xbe\xc7\xda\x05=\xc2\x91a\xbd\xfcS\xb5=\xbb\x8a\x11\xbe\xa6\xb8\xad\xbdi7\xd2<p3,\xbd\xcc\xa3\x8b\xbd\xb52\xb8=\x1bE\x91=dM\xd9=/\x83p\xbd~\xf7\x06\xbe\xe02\x8a=X\xa7\xcc=#+\xeb\xbc\x9c\xc5\xce\xbd\x92\xe9\xf1\xbc\xda0\xd9\xbd\xc6IY\xbb\xd3|\x1b<\xab6\x94\xbd\xb9>C\xbd$\xf1\xfe=\xc7l\x05\xbe\xc4%N\xbb{\x9f\xef\xbc\x9f\xf9\xef<\x02\xc2\xb8=1%\x11\xbc\xd0\xba\x0b\xbe\x8e\x0bF<`\x07\x0b>\x98\xd0\xa2=k\x12\xea\xbd\xcd\x879\xbd\xa1\x14

# Federated Learning

In [15]:
def get_parameters(net) -> List[np.ndarray]:
    parameters = [val.cpu().numpy() for _, val in net.state_dict().items()]
    return parameters


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

## Local Client

In [16]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1, verbose=True)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

In [17]:
def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net().to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader)

In [18]:
from typing import Optional, Dict


def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}


# The `evaluate` function will be by Flower called after every round
def evaluate(server_round: int, parameters: fl.common.NDArrays, config: Dict[str, fl.common.Scalar]) -> Optional[
    Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net()
    valloader = valloaders[0]
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [19]:
Q = 2657003489534545107915232808830590043
fixedPoint = np.vectorize(fixed_point)
floatPoint = np.vectorize(float_point)


class FedAvgSmc(FedAvg):
    def aggregate_fit(
            self,
            rnd: int,
            results: List[Tuple[fl.server.client_proxy.ClientProxy, FitRes]],
            failures: List[BaseException],
    ) -> Optional[Weights]:
        if not results:
            return None

        clients = []
        fit_results = []

        for client, fit_result in results:
            clients.append(client)
            fit_results.append(fit_result)

        fit_res_ndarray_parameters = [parameters_to_ndarrays(result.parameters) for result in
                                      fit_results]  # list of clients, each with a list of layers(each layer representing weights)

        fl_nodes = [VirtualMachine(f"Client: {client.cid}") for client in clients]

        num_layers = len(fit_results[0].parameters.tensors)

        layers_weights = {}

        layers_weights_smc = {}

        for layer in range(num_layers):  #loop through number of layers
            layers_weights[f"layer_{layer}"] = []
            for weights in fit_res_ndarray_parameters:
                layers_weights[f"layer_{layer}"].append(fixedPoint(weights[layer]))

        for layer in range(num_layers):
            fl_node_values = [PrivateScalar(tensor, node) for tensor, node in
                              zip(layers_weights[f'layer_{layer}'], fl_nodes)]

            fl_exchanged_shares = []
            fl_exchanged_shares_list = []
            layers_weights_smc[f"layer_{layer}"] = []

            for value in fl_node_values:
                fl_exchanged_shares.append(value.share_tensor(fl_nodes, Q))

            for client_shares in fl_exchanged_shares:
                fl_exchanged_shares_list = [share.value for share in client_shares.shares]

            layers_weights_smc[f'layer_{layer}'] = floatPoint(fl_exchanged_shares_list)

        for i, client in enumerate(clients):
            client_weights = []
            for layer in layers_weights_smc.values():
                client_weights.append(np.array(layer[i]))
            fit_results[i].parameters = ndarrays_to_parameters(client_weights)

        results = tuple(zip(clients, fit_results))

        return super().aggregate_fit(rnd, results, failures)


In [20]:
# Create FedAvg strategy
strategy = FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=5,  # Never sample less than 5 clients for training
    min_evaluate_clients=3,  # Never sample less than 3 clients for evaluation
    min_available_clients=5,  # Wait until 5 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,  # pass the metric aggregation function
    evaluate_fn=evaluate,  # Pass the evaluation function
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=strategy,
)

INFO flower 2023-03-08 11:28:39,578 | app.py:140 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-03-08 11:28:41,579	INFO worker.py:1518 -- Started a local Ray instance.
INFO flower 2023-03-08 11:28:44,405 | app.py:174 | Flower VCE: Ray initialized with resources: {'object_store_memory': 2263127654.0, 'CPU': 4.0, 'node:127.0.0.1': 1.0, 'memory': 4526255310.0}
INFO flower 2023-03-08 11:28:44,407 | server.py:86 | Initializing global parameters
INFO flower 2023-03-08 11:28:44,408 | server.py:270 | Requesting initial parameters from one random client
INFO flower 2023-03-08 11:28:47,867 | server.py:274 | Received initial parameters from one random client
INFO flower 2023-03-08 11:28:47,868 | server.py:88 | Evaluating initial parameters
INFO flower 2023-03-08 11:28:48,230 | server.py:91 | initial parameters (loss, other metrics): 0.07396003937721253, {'accuracy': 0.099}
INFO flower 2023-03-08 11:28:48,231 | server.py:101 | FL starting
DEBUG flower 20

Server-side evaluation loss 0.07396003937721253 / accuracy 0.099
(launch_and_fit pid=5472) Epoch 1: train loss:  0.061123188585042953, accuracy: 0.2791111111111111, TIME TAKEN: 8.85133171081543
(launch_and_fit pid=13000) Epoch 1: train loss:  0.06156037002801895, accuracy: 0.2767777777777778, TIME TAKEN: 9.355722665786743
(launch_and_fit pid=11596) Epoch 1: train loss:  0.06110134720802307, accuracy: 0.28055555555555556, TIME TAKEN: 9.262115478515625
(launch_and_fit pid=956) Epoch 1: train loss:  0.06066473573446274, accuracy: 0.27944444444444444, TIME TAKEN: 9.303003549575806


DEBUG flower 2023-03-08 11:29:08,756 | server.py:229 | fit_round 1 received 5 results and 0 failures
WARNING flower 2023-03-08 11:29:08,769 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


(launch_and_fit pid=5472) Epoch 1: train loss:  0.06045028194785118, accuracy: 0.27666666666666667, TIME TAKEN: 6.650487422943115


INFO flower 2023-03-08 11:29:09,116 | server.py:116 | fit progress: (1, 0.06169295358657837, {'accuracy': 0.328}, 20.883521700000003)
DEBUG flower 2023-03-08 11:29:09,117 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 5)


...

(launch_and_fit pid=11596) Epoch 1: train loss:  0.03909638151526451, accuracy: 0.5544444444444444, TIME TAKEN: 6.130931854248047


INFO flower 2023-03-08 11:32:58,203 | server.py:116 | fit progress: (10, 0.038344652891159056, {'accuracy': 0.572}, 249.97063740000002)
DEBUG flower 2023-03-08 11:32:58,204 | server.py:165 | evaluate_round 10: strategy sampled 3 clients (out of 5)


Server-side evaluation loss 0.038344652891159056 / accuracy 0.572


DEBUG flower 2023-03-08 11:33:01,634 | server.py:179 | evaluate_round 10 received 3 results and 0 failures
INFO flower 2023-03-08 11:33:01,636 | server.py:144 | FL finished in 253.4029897
INFO flower 2023-03-08 11:33:01,637 | app.py:192 | app_fit: losses_distributed [(1, 0.06130163772900899), (2, 0.05058794663349788), (3, 0.04804842785994212), (4, 0.045576356569925945), (5, 0.04487536136309306), (6, 0.04297814947366715), (7, 0.04189349895715714), (8, 0.04004820213715235), (9, 0.03934662687778473), (10, 0.039831420004367825)]
INFO flower 2023-03-08 11:33:01,639 | app.py:193 | app_fit: metrics_distributed {'accuracy': [(1, 0.3296666666666667), (2, 0.434), (3, 0.46366666666666667), (4, 0.49066666666666664), (5, 0.4923333333333333), (6, 0.5186666666666667), (7, 0.5366666666666667), (8, 0.552), (9, 0.569), (10, 0.552)]}
INFO flower 2023-03-08 11:33:01,641 | app.py:194 | app_fit: losses_centralized [(0, 0.07396003937721253), (1, 0.06169295358657837), (2, 0.05087333846092224), (3, 0.047977032

History (loss, distributed):
	round 1: 0.06130163772900899
	round 2: 0.05058794663349788
	round 3: 0.04804842785994212
	round 4: 0.045576356569925945
	round 5: 0.04487536136309306
	round 6: 0.04297814947366715
	round 7: 0.04189349895715714
	round 8: 0.04004820213715235
	round 9: 0.03934662687778473
	round 10: 0.039831420004367825
History (loss, centralized):
	round 0: 0.07396003937721253
	round 1: 0.06169295358657837
	round 2: 0.05087333846092224
	round 3: 0.04797703206539154
	round 4: 0.0455112464427948
	round 5: 0.04387184917926788
	round 6: 0.04220900583267212
	round 7: 0.04070673787593842
	round 8: 0.0395942370891571
	round 9: 0.038562608480453495
	round 10: 0.038344652891159056
History (metrics, distributed):
{'accuracy': [(1, 0.3296666666666667), (2, 0.434), (3, 0.46366666666666667), (4, 0.49066666666666664), (5, 0.4923333333333333), (6, 0.5186666666666667), (7, 0.5366666666666667), (8, 0.552), (9, 0.569), (10, 0.552)]}History (metrics, centralized):
{'accuracy': [(0, 0.099), (1,

In [21]:
# Create FedAvgSmc strategy
strategy = FedAvgSmc(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=5,  # Never sample less than 5 clients for training
    min_evaluate_clients=3,  # Never sample less than 3 clients for evaluation
    min_available_clients=5,  # Wait until 5 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,  # pass the metric aggregation function
    evaluate_fn=evaluate,  # Pass the evaluation function
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=strategy,
)

INFO flower 2023-03-08 11:33:01,661 | app.py:140 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-03-08 11:33:05,484	INFO worker.py:1518 -- Started a local Ray instance.
INFO flower 2023-03-08 11:33:08,649 | app.py:174 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'CPU': 4.0, 'object_store_memory': 2339804774.0, 'memory': 4679609550.0}
INFO flower 2023-03-08 11:33:08,651 | server.py:86 | Initializing global parameters
INFO flower 2023-03-08 11:33:08,652 | server.py:270 | Requesting initial parameters from one random client
INFO flower 2023-03-08 11:33:11,836 | server.py:274 | Received initial parameters from one random client
INFO flower 2023-03-08 11:33:11,837 | server.py:88 | Evaluating initial parameters
INFO flower 2023-03-08 11:33:12,199 | server.py:91 | initial parameters (loss, other metrics): 0.07363504862785339, {'accuracy': 0.094}
INFO flower 2023-03-08 11:33:12,199 | server.py:101 | FL starting
DEBUG flower 20

Server-side evaluation loss 0.07363504862785339 / accuracy 0.094
(launch_and_fit pid=2424) Epoch 1: train loss:  0.061650294810533524, accuracy: 0.264, TIME TAKEN: 9.54763674736023
(launch_and_fit pid=8832) Epoch 1: train loss:  0.06143491715192795, accuracy: 0.2842222222222222, TIME TAKEN: 9.156908512115479
(launch_and_fit pid=9872) Epoch 1: train loss:  0.06108034774661064, accuracy: 0.2838888888888889, TIME TAKEN: 9.18016791343689
(launch_and_fit pid=13340) Epoch 1: train loss:  0.06139653921127319, accuracy: 0.2733333333333333, TIME TAKEN: 9.085937976837158


DEBUG flower 2023-03-08 11:33:33,700 | server.py:229 | fit_round 1 received 5 results and 0 failures


(launch_and_fit pid=2424) Epoch 1: train loss:  0.06221187859773636, accuracy: 0.2633333333333333, TIME TAKEN: 6.843791723251343


WARNING flower 2023-03-08 11:33:35,548 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
INFO flower 2023-03-08 11:33:35,896 | server.py:116 | fit progress: (1, nan, {'accuracy': 0.101}, 23.697069999999997)
DEBUG flower 2023-03-08 11:33:35,898 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 5)


...

(launch_and_fit pid=2424) Epoch 1: train loss:  nan, accuracy: 0.10255555555555555, TIME TAKEN: 6.964486598968506


INFO flower 2023-03-08 11:37:29,726 | server.py:116 | fit progress: (10, nan, {'accuracy': 0.101}, 257.52619630000004)
DEBUG flower 2023-03-08 11:37:29,726 | server.py:165 | evaluate_round 10: strategy sampled 3 clients (out of 5)


Server-side evaluation loss nan / accuracy 0.101


DEBUG flower 2023-03-08 11:37:33,548 | server.py:179 | evaluate_round 10 received 3 results and 0 failures
INFO flower 2023-03-08 11:37:33,550 | server.py:144 | FL finished in 261.35029369999995
INFO flower 2023-03-08 11:37:33,552 | app.py:192 | app_fit: losses_distributed [(1, nan), (2, nan), (3, nan), (4, nan), (5, nan), (6, nan), (7, nan), (8, nan), (9, nan), (10, nan)]
INFO flower 2023-03-08 11:37:33,554 | app.py:193 | app_fit: metrics_distributed {'accuracy': [(1, 0.09266666666666666), (2, 0.09133333333333334), (3, 0.09266666666666667), (4, 0.09233333333333334), (5, 0.08766666666666667), (6, 0.09633333333333334), (7, 0.09266666666666667), (8, 0.10233333333333335), (9, 0.09633333333333334), (10, 0.08266666666666667)]}
INFO flower 2023-03-08 11:37:33,555 | app.py:194 | app_fit: losses_centralized [(0, 0.07363504862785339), (1, nan), (2, nan), (3, nan), (4, nan), (5, nan), (6, nan), (7, nan), (8, nan), (9, nan), (10, nan)]
INFO flower 2023-03-08 11:37:33,557 | app.py:195 | app_fit: m

History (loss, distributed):
	round 1: nan
	round 2: nan
	round 3: nan
	round 4: nan
	round 5: nan
	round 6: nan
	round 7: nan
	round 8: nan
	round 9: nan
	round 10: nan
History (loss, centralized):
	round 0: 0.07363504862785339
	round 1: nan
	round 2: nan
	round 3: nan
	round 4: nan
	round 5: nan
	round 6: nan
	round 7: nan
	round 8: nan
	round 9: nan
	round 10: nan
History (metrics, distributed):
{'accuracy': [(1, 0.09266666666666666), (2, 0.09133333333333334), (3, 0.09266666666666667), (4, 0.09233333333333334), (5, 0.08766666666666667), (6, 0.09633333333333334), (7, 0.09266666666666667), (8, 0.10233333333333335), (9, 0.09633333333333334), (10, 0.08266666666666667)]}History (metrics, centralized):
{'accuracy': [(0, 0.094), (1, 0.101), (2, 0.101), (3, 0.101), (4, 0.101), (5, 0.101), (6, 0.101), (7, 0.101), (8, 0.101), (9, 0.101), (10, 0.101)]}